In [22]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression 
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
import re 
import string

data = pd.read_csv(r"C:\Users\KIIT\Desktop\mini project\fake reviews dataset.csv")
print(data.shape)
data.head()



(40432, 4)


,category,rating,label,text_
0,Home_and_Kitchen_5,5.0,CG,"Love this! Well made, sturdy, and very comfor..."
1,Home_and_Kitchen_5,5.0,CG,"love it, a great upgrade from the original. I..."
2,Home_and_Kitchen_5,5.0,CG,This pillow saved my back. I love the look and...
3,Home_and_Kitchen_5,1.0,CG,"Missing information on how to use it, but it i..."
4,Home_and_Kitchen_5,5.0,CG,Very nice set. Good quality. We have had the s...


In [2]:
print(data['category'].unique())
data['label_num'] = data.category.map({
    'Home_and_Kitchen_5' : 0,
    'Sports_and_Outdoors_5' : 1,
    'Electronics_5' : 2,
    'Movies_and_TV_5' : 3,
    'Tools_and_Home_Improvement_5' : 4,
    'Pet_Supplies_5' : 5,
    'Kindle_Store_5' : 6,
    'Books_5' : 7,
    'Toys_and_Games_5' : 8,
    'Clothing_Shoes_and_Jewelry_5' : 9
})
data.head()
print(data[data['label_num'].isna()])  # Display rows with NaN in 'label_num'

['Home_and_Kitchen_5' 'Sports_and_Outdoors_5' 'Electronics_5'
 'Movies_and_TV_5' 'Tools_and_Home_Improvement_5' 'Pet_Supplies_5'
 'Kindle_Store_5' 'Books_5' 'Toys_and_Games_5'
 'Clothing_Shoes_and_Jewelry_5']
Empty DataFrame
Columns: [category, rating, label, text_, label_num]
Index: []


In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    data.text_,
    data.label_num,
    test_size=0.2, #using 20% of the data as a test dataset
    random_state=19038,
    stratify=data.label_num
)

print(X_train.shape)
print(X_test.shape)

(32345,)
(8087,)


In [6]:
vector = TfidfVectorizer()

X_train_vector = vector.fit_transform(X_train)
X_test_vector = vector.transform(X_test)

In [ ]:
model = LogisticRegression(max_iter=45000)
model.fit(X_train_vector, y_train)

lr_prediction = model.predict(X_test_vector)
print("Lr accuracy is = ", (metrics.accuracy_score(lr_prediction, y_test))*100)

#Logistic regression : 79.26 (max iterations reached)
#Multinomial naive bayes : 76.9
#Stochastic gradient descent : 79.27
#RandomForestClassifier : 71.97 (after 2m 16.8s)
#Bernouli Naive Bayes : 75.46



Lr accuracy is =  65.2528749845431


c:\Users\KIIT\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
